In [1]:
from datasets import load_dataset,DatasetDict,Dataset
import numpy as np
from transformers import AutoFeatureExtractor
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

In [2]:
# import random
# import shutil
# import os
# dir_path=os.getcwd()+"\\KTH-TIPS2-b\\"
# res={}
# folders=os.listdir(dir_path)
# for x in folders:
#     for path in os.listdir(os.path.join(dir_path,x)):
#         if x in res:
#             res[x].append(path)
#         else:
#             res[x]=[path]
# test={}
# train={}
# print([len(res[x]) for x in res])
# for x in res:
#     test_temp=random.sample(res[x],38)
#     train_temp=[y for y in res[x] if y not in test_temp]
#     test[x]=test_temp
#     train[x]=train_temp
# for x in train:
#     dir=os.path.join(dir_path,x)
#     os.mkdir(dir_path+"train\\"+x)
#     for y in train[x]:
#         shutil.move(dir+"\\"+y,dir_path+"train\\"+x+"\\"+y)
# for x in test:
#     dir=os.path.join(dir_path,x)
#     os.mkdir(dir_path+"test\\"+x)
#     for y in test[x]:
#         shutil.move(dir+"\\"+y,dir_path+"test\\"+x+"\\"+y)

In [3]:
dataset = load_dataset("imagefolder", data_dir="KTH-TIPS2-b/")
dataset

Resolving data files:   0%|          | 0/4335 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/417 [00:00<?, ?it/s]

Using custom data configuration default-a3732dabccd97050
Reusing dataset imagefolder (/home/parth/.cache/huggingface/datasets/imagefolder/default-a3732dabccd97050/0.0.0/48efdc62d40223daee675ca093d163bcb6cb0b7d7f93eb25aebf5edca72dc597)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 4335
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 417
    })
})

In [4]:
labels = dataset["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [5]:
feature_extractor = AutoFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")

Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [6]:
normalize = Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
_transforms = Compose([RandomResizedCrop(feature_extractor.size), ToTensor(), normalize])

In [7]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples

In [8]:
dataset=dataset.with_transform(transforms)

Parameter 'transform'=<function transforms at 0x7f893665d550> of the transform datasets.arrow_dataset.Dataset.set_format couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


In [9]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

2022-07-13 10:45:25.981277: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-07-13 10:45:25.984324: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-13 10:45:25.984336: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    evaluation_strategy="steps",
    num_train_epochs=4,
    fp16=True,
    save_steps=100,
    eval_steps=100,
    logging_steps=10,
    learning_rate=2e-4,
    save_total_limit=2,
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=food["train"],
    eval_dataset=food["test"],
    tokenizer=feature_extractor,
)

trainer.train()

In [11]:
import torch
torch.cuda.is_available()

False

In [12]:
!nvidia-smi

Wed Jul 13 01:37:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.78       Driver Version: 512.78       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   43C    P8     6W /  N/A |    211MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [15]:
!pip install pytorch torchvision torchaudio cudatoolkit==11.6

  Using cached pytorch-1.0.2.tar.gz (689 bytes)


ERROR: Could not find a version that satisfies the requirement cudatoolkit==11.6
ERROR: No matching distribution found for cudatoolkit==11.6
